<a href="https://colab.research.google.com/github/brunorodriguesti/Experiment_CF_ML_Bug_Fixing_Time/blob/main/Experiment_ML_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [96]:
cd /content/gdrive/My Drive/

/content/gdrive/My Drive


In [97]:
cd Colab\ Notebooks

/content/gdrive/My Drive/Colab Notebooks


In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import nltk
import time
from datetime import timedelta

In [99]:
#Select dataset
#bugfixclass = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/eclipse_clean.csv', low_memory=False)
bugfixclass = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/netbeans_clean.csv', low_memory=False)

In [100]:
bugfixclass['classification'].value_counts()

0.0    56952
1.0    56952
Name: classification, dtype: int64

#Treating the textual features

In [7]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
nltk.download("stopwords")
nltk.download('wordnet')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [8]:
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    #lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(stem_words)

In [ ]:
bugfixclass['summary_plus_comment']= bugfixclass['short_desc']+" " +bugfixclass['comments']

In [ ]:
bugfixclass['summary_plus_comment_tok']= np.array(bugfixclass['summary_plus_comment'].map(lambda s:preprocess(s)))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
countVect = CountVectorizer(min_df=0.005)

In [ ]:
sentence_count_vect = countVect.fit_transform(bugfixclass['summary_plus_comment_tok'])

In [ ]:
 sentence_count_vect.shape

(113904, 1262)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(sentence_count_vect)

In [ ]:
sentence = X_train_tfidf

In [ ]:
sentence.shape

(113904, 1262)

In [ ]:
count_vect_df = pd.DataFrame(sentence.todense(), columns=countVect.get_feature_names())

#Features

In [101]:
features = ['reporter', 'assigned_to', 'priority_n', 'severity_n', 'product_id', 'component_id', 'month_creation', 'year_creation', 'number_cc','number_attachment' ]

In [ ]:
bugfixclass.columns

Index(['Unnamed: 0', 'bug_id', 'assigned_to', 'bug_severity', 'priority',
       'bug_status', 'creation_ts', 'delta_ts', 'short_desc', 'op_sys',
       'rep_platform', 'reporter', 'version', 'resolution', 'lastdiffed',
       'estimated_time', 'product_id', 'component_id', 'short_desc.1',
       'comments', 'days', 'year_creation', 'year_creation_n',
       'month_creation', 'year_lastdiffed', 'month_lastdiffed',
       'classification', 'number_cc', 'number_attachment', 'priority_n',
       'severity_n'],
      dtype='object')

In [102]:
# Using textual information
#X = pd.concat([bugfixclass[features], count_vect_df], axis=1)

# It does not use textual information
X = bugfixclass[features]

In [103]:
y = bugfixclass['classification']

In [104]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import metrics

#Cross validation

In [105]:
def cross_validation(X, y, c, model,v=0):
  return {'precision': cross_val_score(model, X, y, scoring="precision", cv=c, verbose=v), 'recall': cross_val_score(model, X, y, scoring="recall", cv=c, verbose = v), 'f1_score': cross_val_score(model, X, y, scoring="f1", cv=c, verbose=v)};  


In [106]:
kf = KFold(n_splits=10,shuffle=True)

#Naive Bayes

In [107]:
from sklearn.naive_bayes import GaussianNB
#from sklearn.naive_bayes import MultinomialNB

In [108]:
gnb = GaussianNB()

# Used to classify with textual features
#classifier = MultinomialNB() 

In [109]:
start = time.time()
nbScore = cross_validation(X, y, kf, gnb)
#nbScore = cross_validation(XsD, y, kf, gnb)
end = time.time()
print('elapsed time: ', timedelta(seconds= end - start))
print ('Precision: ', round(nbScore['precision'].mean(),2)*100, ' recall: ', round(nbScore['recall'].mean(),2)*100, ' f-measure: ', round(nbScore['f1_score'].mean(),2)*100)

elapsed time:  0:00:01.383014
Precision:  57.99999999999999  recall:  44.0  f-measure:  50.0


#Decision Tree

In [110]:
from sklearn.tree import DecisionTreeClassifier 

In [111]:
cdt = DecisionTreeClassifier(criterion='gini', max_depth=None)

In [112]:
tart = time.time()
dtScore = cross_validation(X, y, kf, cdt)
end = time.time()
print('elapsed time: ', timedelta(seconds=end - start))
print ('Precision: ', round(dtScore['precision'].mean(),2)*100, ' recall: ', round(dtScore['recall'].mean(),2)*100, ' f-measure: ', round(dtScore['f1_score'].mean(),2)*100)

elapsed time:  0:02:18.713404
Precision:  67.0  recall:  65.0  f-measure:  66.0


#KNN

In [113]:
from sklearn.neighbors import KNeighborsClassifier

In [114]:
knn = KNeighborsClassifier()

In [115]:
start = time.time()
knnScore = cross_validation(X, y, kf, knn)
end = time.time()
print('elapsed time: ', timedelta(seconds=end - start))
print ('Precision: ', round(knnScore['precision'].mean(),2)*100, ' recall: ', round(knnScore['recall'].mean(),2)*100, ' f-measure: ', round(knnScore['f1_score'].mean(),2)*100)

elapsed time:  0:00:20.196745
Mean results feature + summary+ comments - eclipse
Precision:  65.0  recall:  62.0  f-measure:  63.0


#Random Forest

In [116]:
from sklearn.ensemble import RandomForestClassifier

In [117]:
rfc=RandomForestClassifier(n_estimators=100)

In [118]:
start = time.time()
rfcScore = cross_validation(X, y, kf, rfc)
end = time.time()
print('elapsed time: ', timedelta(seconds=end - start))
print ('Precision: ', round(rfcScore['precision'].mean(),2)*100, ' recall: ', round(rfcScore['recall'].mean(),2)*100, ' f-measure: ', round(rfcScore['f1_score'].mean(),2)*100)

elapsed time:  0:07:37.072081
Mean results feature + summary+ comments - eclispe
Precision:  72.0  recall:  70.0  f-measure:  71.0


In [119]:
randonForestScore = np.array(rfcScore['f1_score'])

#SVM

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
svmclf = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)

In [ ]:
start = time.time()
svmScore = cross_validation(X, y, kf, svmclf)
end = time.time()
print('elapsed time: ',timedelta(seconds=end - start))
print("Mean results feature + summary+ comments - Eclipse")
print ('Precision: ', round(svmScore['precision'].mean(),2)*100, ' recall: ', round(svmScore['recall'].mean(),2)*100, ' f-measure: ', round(svmScore['f1_score'].mean(),2)*100)

#Collaborative Filtering

In [120]:
first = bugfixclass['days'].mode()
second = np.percentile(bugfixclass['days'], [25])
third = np.percentile(bugfixclass['days'], [50])
forth = np.percentile(bugfixclass['days'], [75])
fifth= np.percentile(bugfixclass['days'], [])
np.percentile(bugfixclass['days'], [25, 50, 75])

array([ 6. , 25.5, 78. ])

In [121]:
bugfixclass.loc[(bugfixclass['days'] <= int(first)), 'ratings'] = 1
bugfixclass.loc[(bugfixclass['days'] > int(first)) & (bugfixclass['days'] <= int(second)) , 'ratings'] = 2
bugfixclass.loc[(bugfixclass['days'] > int(second)) & (bugfixclass['days'] <= int(third)), 'ratings'] = 3
bugfixclass.loc[(bugfixclass['days'] > int(third)) & (bugfixclass['days'] <= int(forth)), 'ratings'] = 4
bugfixclass.loc[bugfixclass['days'] > int(forth), 'ratings'] = 5

In [122]:
#bugfixclass['characteristic']= bugfixclass['bug_severity']+bugfixclass['priority']+bugfixclass['product_id'].astype(str)+bugfixclass['component_id'].astype(str)+bugfixclass['op_sys']+bugfixclass['rep_platform']+bugfixclass['number_cc'].astype(str)+bugfixclass['number_attachment'].astype(str)+bugfixclass['month_creation'].astype(str)+bugfixclass['year_creation'].astype(str)
bugfixclass['characteristic']= bugfixclass['bug_severity']+bugfixclass['priority']+bugfixclass['product_id'].astype(str)+bugfixclass['component_id'].astype(str)+bugfixclass['op_sys']+bugfixclass['rep_platform']

In [123]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [124]:
bugfixclass['characteristic_n'] = le.fit_transform(bugfixclass['characteristic'].values.reshape(-1,1))

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [125]:
def precision_recall_at_k(predictions, k = 10, threshold = 3):
    
    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        est = round(est, 0)
        user_est_true[uid].append((est, true_r))
        #print ('estimado '+ str(est)+ ' verdadeiro: '+ str(true_r))
    
    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():
        
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        
        # Number of relevant items
        n_rel = sum((true_r <= threshold) for (_, true_r) in user_ratings)
        
        # Number of recommended items in top k
        n_rec_k = sum((est <= threshold) for (est, _) in user_ratings[:k])
        
        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r <= threshold) and (est <= threshold)) for (est, true_r) in user_ratings[:k])
        
        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1
        
        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1
        #print ("ratings: "+ str(user_ratings) + " predicted: "+ str(user_est_true) + ' true_r '+ str(true_r))
    
    return precisions, recalls;

In [126]:
def get_precision_vs_recall(algo, k_max = 10, verbose = False):
    precision_list = []
    recall_list = []
    f1_score_list = []
        
    if algo:
        for k_curr in range(1, k_max + 1):
            algo.fit(data_train)
            predictions = algo.test(data_testset)
           # print("prediction ", predictions)
            
            # Get precision and recall at k metrics for each user
            precisions, recalls = precision_recall_at_k(predictions, k = k_curr, threshold = 3)
            
            # Precision and recall can then be averaged over all users
            precision = sum(prec for prec in precisions.values()) / len(precisions)
            recall = sum(rec for rec in recalls.values()) / len(recalls)
            f1_score = 2 * (precision * recall) / (precision + recall)
            
            # Save measures
            precision_list.append(precision)
            recall_list.append(recall)
            f1_score_list.append(f1_score)

            
            
            if verbose:
                print('K =', k_curr, '- Precision:', precision, ', Recall:', recall, ', F1 score:', f1_score)
    
    return {'precision': precision_list, 'recall': recall_list, 'f1_score': f1_score_list};

In [26]:
pip install scikit-surprise

In [127]:
features = ['assigned_to','reporter', 'characteristic_n', 'ratings' ]
data = bugfixclass[features]

In [128]:
from surprise import SVD, SVDpp, SlopeOne, CoClustering, NMF, BaselineOnly, NormalPredictor
from surprise import Reader
from surprise import Dataset
from surprise import accuracy
from surprise.prediction_algorithms import KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore
from surprise.model_selection import GridSearchCV
#from surprise.model_selection import train_test_split
from sklearn.model_selection import train_test_split
import timeit
import time
from collections import defaultdict
from surprise import accuracy
from surprise import dump

In [129]:
reader = Reader(rating_scale=(1, 5))
trainset, testset = train_test_split(data, test_size=0.20)

In [130]:
reader = Reader(rating_scale = (1, 5))
data_train = Dataset.load_from_df(trainset[['reporter', 'characteristic_n', 'ratings']], reader)
data_test = Dataset.load_from_df(testset[['reporter', 'characteristic_n', 'ratings']], reader)

In [131]:
data_train = data_train.build_full_trainset()
data_test = data_test.build_full_trainset()

# Create the trainset and testset
data_trainset = data_train.build_testset()
data_testset = data_test.build_testset()

In [132]:
k_max = 10

#SVD

In [133]:
algo = SVD()
start = time.time()
metrics_SVD = get_precision_vs_recall(algo, k_max, True)
end = time.time()
print ('precision ', round(np.array(metrics_SVD['precision']).mean(),2)*100, 'recall ', round(np.array(metrics_SVD['recall']).mean(),2)*100, ' f1-score ', round(np.array(metrics_SVD['f1_score']).mean(),2)*100, 'elapsed time: ', timedelta(seconds=end - start))

K = 1 - Precision: 0.7994121969140338 , Recall: 0.6194200534266138 , F1 score: 0.6979992815972313
K = 2 - Precision: 0.7667156502571638 , Recall: 0.6647398751547386 , F1 score: 0.7120954253670919
K = 3 - Precision: 0.7544289329741206 , Recall: 0.6869328756429826 , F1 score: 0.7191005524052293
K = 4 - Precision: 0.7444485264103192 , Recall: 0.7019707027410474 , F1 score: 0.7225858792618433
K = 5 - Precision: 0.7418319862845946 , Recall: 0.7082442355767286 , F1 score: 0.7246491186209448
K = 6 - Precision: 0.7403502326720547 , Recall: 0.7170571149025463 , F1 score: 0.7285175318221433
K = 7 - Precision: 0.7298129293353391 , Recall: 0.7213178918031709 , F1 score: 0.7255405452498703
K = 8 - Precision: 0.7298350885786593 , Recall: 0.7302393578155417 , F1 score: 0.7300371672295378
K = 9 - Precision: 0.7273620975861975 , Recall: 0.7342321289472993 , F1 score: 0.7307809674274841
K = 10 - Precision: 0.7292907292723602 , Recall: 0.7372298705256123 , F1 score: 0.7332388102711394
precision  75.0 rec

In [134]:
#To statistic method
svdPrediction = np.array(metrics_SVD['f1_score'])

#KNNBasic

In [135]:
algo = KNNBasic()
start = time.time()
metrics_KNN = get_precision_vs_recall(algo, k_max, True)
end = time.time()
print ('precision ', round(np.array(metrics_KNN['precision']).mean(),2)*100, 'recall ', round(np.array(metrics_KNN['recall']).mean(),2)*100, ' f1-score ', round(np.array(metrics_KNN['f1_score']).mean(),2)*100, 'elapsed time: ', timedelta(seconds=end - start))

Computing the msd similarity matrix...
Done computing similarity matrix.
K = 1 - Precision: 0.6833210874357091 , Recall: 0.6719472572016657 , F1 score: 0.6775864459718334
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 2 - Precision: 0.6383786431545432 , Recall: 0.7301839072090724 , F1 score: 0.6812020565863982
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 3 - Precision: 0.6192750428606419 , Recall: 0.7566478264693676 , F1 score: 0.6811037531419123
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 4 - Precision: 0.6060086537676547 , Recall: 0.7744933527772843 , F1 score: 0.6799695644676786
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 5 - Precision: 0.5995183280267776 , Recall: 0.7845660110051796 , F1 score: 0.6796720255839894
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 6 - Precision: 0.592766756469916 , Recall: 0.7922495575110042 , F1 s

#KNNWithMeans

In [136]:
algo = KNNWithMeans()
start = time.time()
metrics_KNNM = get_precision_vs_recall(algo, k_max, True)
end = time.time()
print ('precision ', round(np.array(metrics_KNNM['precision']).mean(),2)*100, 'recall ', round(np.array(metrics_KNNM['recall']).mean(),2)*100, ' f1-score ', round(np.array(metrics_KNNM['f1_score']).mean(),2)*100, 'elapsed time: ', timedelta(seconds=end - start))

Computing the msd similarity matrix...
Done computing similarity matrix.
K = 1 - Precision: 0.7127112417340191 , Recall: 0.6510271807867286 , F1 score: 0.6804741770983547
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 2 - Precision: 0.6803820720058781 , Recall: 0.7021981511069486 , F1 score: 0.6911179908724241
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 3 - Precision: 0.6641766674830603 , Recall: 0.7250752414329625 , F1 score: 0.6932911942588963
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 4 - Precision: 0.6566250306147443 , Recall: 0.7411325091653114 , F1 score: 0.6963241372989927
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 5 - Precision: 0.6484284431382151 , Recall: 0.7506555651512778 , F1 score: 0.6958072804208115
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 6 - Precision: 0.6441913625602088 , Recall: 0.7590450949014209 , F1 

#KNNWithZScore

In [137]:
algo = KNNWithZScore()
start = time.time()
metrics_KNNZ = get_precision_vs_recall(algo, k_max, True)
end = time.time()
print ('precision ', round(np.array(metrics_KNNZ['precision']).mean(),2)*100, 'recall ', round(np.array(metrics_KNNZ['recall']).mean(),2)*100, ' f1-score ', round(np.array(metrics_KNNZ['f1_score']).mean(),2)*100, 'elapsed time: ', timedelta(seconds=end - start))

Computing the msd similarity matrix...
Done computing similarity matrix.
K = 1 - Precision: 0.7117315699240754 , Recall: 0.6496717957255908 , F1 score: 0.6792871808224346
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 2 - Precision: 0.6813617438158217 , Recall: 0.7010040036978642 , F1 score: 0.6910433237231534
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 3 - Precision: 0.6654012572454897 , Recall: 0.7233449676588504 , F1 score: 0.6931642977975347
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 4 - Precision: 0.6567474895909872 , Recall: 0.7399134877841779 , F1 score: 0.6958543748104451
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 5 - Precision: 0.6488611315209402 , Recall: 0.749114864233895 , F1 score: 0.6953932255231705
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 6 - Precision: 0.6449342803494162 , Recall: 0.7571055995035133 , F1 s

#KNNBaseline

In [138]:
algo = KNNBaseline()
start = time.time()
metrics_KNNBL = get_precision_vs_recall(algo, k_max, True)
end = time.time()
print ('precision ', round(np.array(metrics_KNNBL['precision']).mean(),2)*100, 'recall ', round(np.array(metrics_KNNBL['recall']).mean(),2)*100, ' f1-score ', round(np.array(metrics_KNNBL['f1_score']).mean(),2)*100, 'elapsed time: ', timedelta(seconds=end - start))

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 1 - Precision: 0.7957384276267451 , Recall: 0.6197661457756848 , F1 score: 0.6968140514733924
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 2 - Precision: 0.7566740142052413 , Recall: 0.6694141872924753 , F1 score: 0.7103744631398908
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 3 - Precision: 0.7389174626500123 , Recall: 0.6924206215778563 , F1 score: 0.7149138200411368
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 4 - Precision: 0.7292840231855664 , Recall: 0.7068656376859711 , F1 score: 0.7178998542400068
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 5 - Precision: 0.724120336353988 , Recall: 0.7172349190669529 , F1 score: 0.72066118

#SVDpp

In [139]:
algo = SVDpp()
start = time.time()
metrics_SVDpp = get_precision_vs_recall(algo, k_max, True)
end = time.time()
print ('precision ', round(np.array(metrics_SVDpp['precision']).mean(),2)*100, 'recall ', round(np.array(metrics_SVDpp['recall']).mean(),2)*100, ' f1-score ', round(np.array(metrics_SVDpp['f1_score']).mean(),2)*100, 'elapsed time: ', timedelta(seconds=end - start))

K = 1 - Precision: 0.807739407298555 , Recall: 0.6159597721323845 , F1 score: 0.6989327358618451
K = 2 - Precision: 0.7763899093803576 , Recall: 0.6615787111670282 , F1 score: 0.71440089619677
K = 3 - Precision: 0.7594089313413342 , Recall: 0.6854283046425066 , F1 score: 0.7205245869583748
K = 4 - Precision: 0.754102375704139 , Recall: 0.6932129771877752 , F1 score: 0.7223768502444398
K = 5 - Precision: 0.752522654910605 , Recall: 0.7040484591480184 , F1 score: 0.7274789545805367
K = 6 - Precision: 0.744628132908809 , Recall: 0.7132548196002669 , F1 score: 0.7286039029307462
K = 7 - Precision: 0.7438508099786572 , Recall: 0.718981980675551 , F1 score: 0.7312050045670465
K = 8 - Precision: 0.7358513231400816 , Recall: 0.7228599661218509 , F1 score: 0.7292977937874109
K = 9 - Precision: 0.740900423358175 , Recall: 0.7258999689170014 , F1 score: 0.7333234939378114
K = 10 - Precision: 0.7340281422390165 , Recall: 0.7295691028411981 , F1 score: 0.7317918300183196
precision  75.0 recall  70.

#SlopeOne

In [140]:
algo = SlopeOne()
start = time.time()
metrics_slope = get_precision_vs_recall(algo, k_max, True)
end = time.time()
print ('precision ', round(np.array(metrics_slope['precision']).mean(),2)*100, 'recall ', round(np.array(metrics_slope['recall']).mean(),2)*100, ' f1-score ', round(np.array(metrics_slope['f1_score']).mean(),2)*100, 'elapsed time: ', timedelta(seconds=end - start))

K = 1 - Precision: 0.7178545187362234 , Recall: 0.6480307978685754 , F1 score: 0.6811579725983602
K = 2 - Precision: 0.6819740386970365 , Recall: 0.6990556035448907 , F1 score: 0.6904091825999911
K = 3 - Precision: 0.6637684708955837 , Recall: 0.7212872258806533 , F1 score: 0.6913335255956369
K = 4 - Precision: 0.6510123275369419 , Recall: 0.7378619779416172 , F1 score: 0.6917216939876629
K = 5 - Precision: 0.6449016246224181 , Recall: 0.7477538604621388 , F1 score: 0.6925297528274756
K = 6 - Precision: 0.6380480039186872 , Recall: 0.7554509435113819 , F1 score: 0.6918038473654683
K = 7 - Precision: 0.6351317308701585 , Recall: 0.7619909486033504 , F1 score: 0.6928019095305459
K = 8 - Precision: 0.632323629917311 , Recall: 0.7670490533810954 , F1 score: 0.6932009571822457
K = 9 - Precision: 0.6288152774376139 , Recall: 0.7709692110097647 , F1 score: 0.6926740827864064
K = 10 - Precision: 0.6264817536125397 , Recall: 0.7743560978824643 , F1 score: 0.6926140175384885
precision  65.0 reca

#CoClustering

In [141]:
algo = CoClustering()
start = time.time()
metrics_coc = get_precision_vs_recall(algo, k_max, True)
end = time.time()
print ('precision ', round(np.array(metrics_coc['precision']).mean(),2)*100, 'recall ', round(np.array(metrics_coc['recall']).mean(),2)*100, ' f1-score ', round(np.array(metrics_coc['f1_score']).mean(),2)*100, 'elapsed time: ', timedelta(seconds=end - start))

K = 1 - Precision: 0.7063433749693853 , Recall: 0.6568573331635377 , F1 score: 0.6807021487181953
K = 2 - Precision: 0.6749938770511879 , Recall: 0.7096740346405375 , F1 score: 0.6918996591743406
K = 3 - Precision: 0.6571148665197161 , Recall: 0.7357803113003222 , F1 score: 0.6942262257015487
K = 4 - Precision: 0.6475426565433916 , Recall: 0.7529329392089575 , F1 score: 0.6962723193937163
K = 5 - Precision: 0.6431749530573924 , Recall: 0.7570369373961817 , F1 score: 0.6954764489462751
K = 6 - Precision: 0.6404237080578007 , Recall: 0.7669457718047622 , F1 score: 0.6979975935053289
K = 7 - Precision: 0.6330662561375273 , Recall: 0.7745109342805561 , F1 score: 0.6966818457138177
K = 8 - Precision: 0.6278220962644178 , Recall: 0.7799364106392336 , F1 score: 0.6956609530387052
K = 9 - Precision: 0.630604539146053 , Recall: 0.7803544168717611 , F1 score: 0.6975327458296254
K = 10 - Precision: 0.6237561666841608 , Recall: 0.7908584695987453 , F1 score: 0.697437782324713
precision  65.0 recal

#NMF


In [142]:
algo = NMF()
start = time.time()
metrics_nmf = get_precision_vs_recall(algo, k_max, True)
end = time.time()
print ('precision ', round(np.array(metrics_nmf['precision']).mean(),2)*100, 'recall ', round(np.array(metrics_nmf['recall']).mean(),2)*100, ' f1-score ', round(np.array(metrics_nmf['f1_score']).mean(),2)*100, 'elapsed time: ', timedelta(seconds=end - start))

K = 1 - Precision: 0.6857702669605682 , Recall: 0.6704888774159491 , F1 score: 0.6780434821267161
K = 2 - Precision: 0.6484202792064658 , Recall: 0.7293593477287152 , F1 score: 0.6865123894277889
K = 3 - Precision: 0.6335619234223201 , Recall: 0.752241819559552 , F1 score: 0.6878200127434495
K = 4 - Precision: 0.62252020573108 , Recall: 0.7703066348093627 , F1 score: 0.688572952243616
K = 5 - Precision: 0.6103437015266551 , Recall: 0.781519749680836 , F1 score: 0.685405822564815
K = 6 - Precision: 0.6068577026696057 , Recall: 0.7871909526136 , F1 score: 0.6853604302187555
K = 7 - Precision: 0.596928029110248 , Recall: 0.7963885015339888 , F1 score: 0.6823813658579649
K = 8 - Precision: 0.5989471443733015 , Recall: 0.7996816967432802 , F1 score: 0.6849094693194021
K = 9 - Precision: 0.5937989106982492 , Recall: 0.8089150639967458 , F1 score: 0.6848621921701684
K = 10 - Precision: 0.5909437699481783 , Recall: 0.8118157829064777 , F1 score: 0.6839910350678509
precision  62.0 recall  77.0 

#BaselineOnly



In [143]:
algo = BaselineOnly()
start = time.time()
metrics_bl = get_precision_vs_recall(algo, k_max, True)
end = time.time()
print ('precision ', round(np.array(metrics_bl['precision']).mean(),2)*100, 'recall ', round(np.array(metrics_bl['recall']).mean(),2)*100, ' f1-score ', round(np.array(metrics_bl['f1_score']).mean(),2)*100, 'elapsed time: ', timedelta(seconds=end - start))

Estimating biases using als...
K = 1 - Precision: 0.7795738427626745 , Recall: 0.6323280042190264 , F1 score: 0.6982728625070889
Estimating biases using als...
K = 2 - Precision: 0.7444281165809454 , Recall: 0.6838677377343626 , F1 score: 0.7128640336719448
Estimating biases using als...
K = 3 - Precision: 0.7280186137643894 , Recall: 0.707511876048459 , F1 score: 0.7176187742132782
Estimating biases using als...
K = 4 - Precision: 0.7216099273410075 , Recall: 0.722872579882407 , F1 score: 0.7222407017561353
Estimating biases using als...
K = 5 - Precision: 0.71851987917381 , Recall: 0.7333754287528625 , F1 score: 0.7258716542159402
Estimating biases using als...
K = 6 - Precision: 0.713237815331864 , Recall: 0.7402236437602805 , F1 score: 0.7264802120894818
Estimating biases using als...
K = 7 - Precision: 0.7101932519272711 , Recall: 0.7459019950431522 , F1 score: 0.7276097694582966
Estimating biases using als...
K = 8 - Precision: 0.7082534434297844 , Recall: 0.7511066417759592 , F1

#NormalPredictor

In [144]:
algo = NormalPredictor()
start = time.time()
metrics_norm = get_precision_vs_recall(algo, k_max, True)
end = time.time()
print ('precision ', round(np.array(metrics_norm['precision']).mean(),2)*100, 'recall ', round(np.array(metrics_norm['recall']).mean(),2)*100, ' f1-score ', round(np.array(metrics_norm['f1_score']).mean(),2)*100, 'elapsed time: ', timedelta(seconds=end - start))

K = 1 - Precision: 0.7950036737692873 , Recall: 0.5689525675565352 , F1 score: 0.6632461771181761
K = 2 - Precision: 0.7418564780798432 , Recall: 0.6088660298799902 , F1 score: 0.6688142174094253
K = 3 - Precision: 0.710139603232917 , Recall: 0.6299949655754209 , F1 score: 0.6676708224761094
K = 4 - Precision: 0.6866682994530164 , Recall: 0.649327933475619 , F1 score: 0.6674762950336279
K = 5 - Precision: 0.6693689280757611 , Recall: 0.6516720598094587 , F1 score: 0.6604019589579626
K = 6 - Precision: 0.6664952240999263 , Recall: 0.662971606726199 , F1 score: 0.6647287459173263
K = 7 - Precision: 0.6610772891081489 , Recall: 0.6789060790897676 , F1 score: 0.6698730760028607
K = 8 - Precision: 0.6566483561340285 , Recall: 0.6718298906579582 , F1 score: 0.664152377906913
K = 9 - Precision: 0.6622832184551507 , Recall: 0.689050913322927 , F1 score: 0.6754019540000971
K = 10 - Precision: 0.6504584436436016 , Recall: 0.6833528296758191 , F1 score: 0.6665000168190024
precision  69.0 recall  

# Wilcoxon Test

In [92]:
from scipy.stats import wilcoxon, friedmanchisquare, rankdata

In [93]:
wilcoxon(randonForestScore, svdPrediction, zero_method='zsplit')

WilcoxonResult(statistic=1.0, pvalue=0.0069104298078147995)

In [94]:
stat, p = wilcoxon(randonForestScore, svdPrediction)
print('stat=%.3f, p=%.3f' % (stat, p))
if p >= 0.05:
	print('statistically significant')
else:
	print('It is not statistically significant')

stat=1.000, p=0.007
It is not statistically significant
